## Predict strength of Concrete
A prototype API for predicting high performance concrete.

In [1]:
#Import necessary libraries
import flask
from flask import Flask, render_template, request,jsonify
import pickle
import numpy as np
from scipy import misc
import os
import pandas as pd

In [2]:
#define routes
app = Flask(__name__)

@app.route('/', methods=['GET'])
def home():
    return '''<h1>Predict strength of Concrete</h1>
<p>A prototype API for predicting high performance concrete.</p>'''

@app.route('/predict', methods=['POST'])
def apicall():
    """API Call
    Pandas dataframe (sent as a payload) from API Call
    """
    try:
        test_json = request.get_json()
        test = pd.read_json(test_json, orient='records')

        #To resolve the issue of TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'
        #test['Dependents'] = [str(x) for x in list(test['Dependents'])]

        #Getting the Loan_IDs separated out
        cement = test['cement']

    except Exception as e:
        raise e

    clf = 'finalized_model.pkl'

    if test.empty:
        return(bad_request())
    else:
        #Load the saved model
        print("Loading the model...")
        loaded_model = None
        """
        with open('./models/'+clf,'rb') as f:
            loaded_model = pickle.load(f)
        """
        loaded_model = pickle.load(open(filename, 'rb'))
        print("The model has been loaded...doing predictions now...")
        predictions = loaded_model.predict(test)

        """Add the predictions as Series to a new pandas dataframe
                                OR
           Depending on the use-case, the entire test data appended with the new files
        """
        prediction_series = list(pd.Series(predictions))

        final_predictions = pd.DataFrame(list(zip(cement, prediction_series)))

        """We can be as creative in sending the responses.
           But we need to send the response codes as well.
        """
        responses = jsonify(predictions=final_predictions.to_json(orient="records"))
        responses.status_code = 200

        return (responses)

In [ ]:
#Load model from saved file and run the api
if __name__ == '__main__':
    # load the model from disk
    filename = 'model/finalized_model.pkl'
    model = pickle.load(open(filename, 'rb'))
    #app.run(host='0.0.0.0', port=8000, debug=True)
    app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Oct/2019 10:26:21] "GET / HTTP/1.1" 200 -
